In [2]:
import numpy as np

# Define the sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Define the neural network class
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights and biases
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        self.bias_hidden = np.zeros((1, hidden_size))
        self.weights_hidden_output = np.random.rand(hidden_size, output_size)
        self.bias_output = np.zeros((1, output_size))

    def forward(self, inputs):
        # Forward propagation
        self.hidden_input = np.dot(inputs, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = sigmoid(self.hidden_input)
        self.output = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        return sigmoid(self.output)

    def mse_loss(self, predictions, targets):
        return np.mean((targets - predictions) ** 2)

    def get_parameters(self):
        return np.concatenate([
            self.weights_input_hidden.flatten(),
            self.bias_hidden.flatten(),
            self.weights_hidden_output.flatten(),
            self.bias_output.flatten()
        ])

    def set_parameters(self, parameters):
        input_hidden_end = self.weights_input_hidden.size
        hidden_bias_end = input_hidden_end + self.bias_hidden.size
        hidden_output_end = hidden_bias_end + self.weights_hidden_output.size

        self.weights_input_hidden = parameters[:input_hidden_end].reshape(self.weights_input_hidden.shape)
        self.bias_hidden = parameters[input_hidden_end:hidden_bias_end].reshape(self.bias_hidden.shape)
        self.weights_hidden_output = parameters[hidden_bias_end:hidden_output_end].reshape(self.weights_hidden_output.shape)
        self.bias_output = parameters[hidden_output_end:].reshape(self.bias_output.shape)

# Define the genetic algorithm class
class GeneticAlgorithm:
    def __init__(self, population_size, mutation_rate):
        self.population_size = population_size
        self.mutation_rate = mutation_rate

    def initialize_population(self, chromosome_length):
        return [np.random.rand(chromosome_length) for _ in range(self.population_size)]

    def crossover(self, parent1, parent2):
        crossover_point = np.random.randint(0, len(parent1))
        child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        return child

    def mutate(self, child):
        mutation_mask = np.random.rand(len(child)) < self.mutation_rate
        child[mutation_mask] += np.random.uniform(-0.5, 0.5, np.count_nonzero(mutation_mask))
        return child

# Example usage
# Assuming you have input data `X` and target data `y`
# Make sure to normalize your input data before training
# (e.g., dividing by the maximum value or using z-score normalization)

# Define input and target data
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

input_size = 2
hidden_size = 4
output_size = 1
chromosome_length = (input_size * hidden_size) + hidden_size + (hidden_size * output_size) + output_size

# Create a neural network
nn = NeuralNetwork(input_size, hidden_size, output_size)

# Create a genetic algorithm
population_size = 10
mutation_rate = 0.1
genetic_algorithm = GeneticAlgorithm(population_size, mutation_rate)

# Initialize population
population = genetic_algorithm.initialize_population(chromosome_length)

# Train the neural network using genetic algorithm
generations = 10
for generation in range(generations):
    print(f"Generation {generation + 1}/{generations}")

    # Evaluate and select parents
    fitness_scores = []
    for chromosome in population:
        nn.set_parameters(chromosome)
        predictions = nn.forward(X)
        fitness = 1 / nn.mse_loss(predictions, y)  # Fitness is inverse of MSE
        fitness_scores.append((chromosome, fitness))

    fitness_scores.sort(key=lambda x: x[1], reverse=True)
    parents = [chromosome for chromosome, _ in fitness_scores[:2]]

    # Crossover and mutate to create new generation
    new_population = []
    for _ in range(population_size - 2):
        parent1, parent2 = np.random.choice(parents, size=2, replace=False)
        child = genetic_algorithm.crossover(parent1, parent2)
        child = genetic_algorithm.mutate(child)
        new_population.append(child)

    # Elitism: Keep the two best individuals from the previous generation
    new_population.extend(parents)

    population = new_population

# Select the best individual from the final generation
best_chromosome, _ = max(fitness_scores, key=lambda x: x[1])
nn.set_parameters(best_chromosome)

# Test the trained network
predictions = nn.forward(X)
print("Predictions:")
print(predictions)


Generation 1/10


ValueError: a must be 1-dimensional